Functional programming means thinking in terms of mathematical functions. Strict functional programming languages, such as Haskell, provide abstractions for functions which a very similar to their mathematical counterparts. However, most programming languages do not do this so well. Does this mean that the idea of modelling our abstractions in terms of mathematical functions is inappropriate when using these languages ?

A basic premise of this course is that, at least for many modern programming languages, this is not the case. The key to making this approach work is to understand the differences between a mathematical function and a function within the language being used, and then find effective ways of working around these differences.  

What then are the minimal requirements of a programming language for it to be able to support the use functional patterns ? The primary requirement is support for **Higher Order Programming**.



## Higher Order Programming

## Currying

<figure>
<img src="./HaskellBCurry.jpg" style="height:20% display:inline-block">
<img src="./curry.jpeg" style="height:20% display:inline-block">
<figcaption align = "center"> Curry - a popular dish (left). Haskell B Curry - a computer scientist (right). </figcaption>
</figure>

In mathematics a function $f$ can be completely specified in terms of its graph $\Gamma_{f}$. The graph is a (possibly infinite) set of ordered pairs. For example

$\Gamma_{f} = \left\{  (1,1),(2,4),(3,9) \right\}$

which conceptually maps the first element of a given pair to the second. This is emphasised through the notation $f(1) = 1$ and $f(3) = 9$ and so on. 


### Exercise

Give a small example of a graph of function of the form $f(x,y)$.

Write a function in python that implements it.

How many "arguments" does the mathematical function "take" ? 

How mant "arguments" does your python function "take" ?


So, mathematical functions map single value to single values. To make python functions behave in this way they can be **curried**. Here is an example.

In [10]:
from pymonad.tools import curry    

In [9]:
@curry(2)
def f(x,y) :
    return x + y

In [12]:
z = f(2,3)  # function taking two arguments
print(z)

5


In [13]:
g = f(2) # f is a function of one argument that returns a function 
z = g(3)
print(z)

5


In [14]:
z = f(2)(3)
print(z)

5


The "**curry**" pattern is slightly odd in that functional programming languages do not need it - all functions in a functional programming language are automatically "**curried**".

### Exercise 

How can currying be put to practical use ?

### Solution

Currying allows a programmer to provide a way of adding extra parameters to a function whilst still allowing it to be **Reused** in other code. A strategy in the pig game provides an excellent example of a use case for the **curry** pattern. 

### Exercise

Try and use **curried** strategies in your pig competition code.

In [6]:
def f(x,y=2) : return x*y

In [2]:
import inspect
    
    

In [7]:
sig = inspect.signature(f)
for name, param in sig.parameters.items():
    print(f"{name}: {param.kind}, Default: {param.default}")

x: POSITIONAL_OR_KEYWORD, Default: <class 'inspect._empty'>
y: POSITIONAL_OR_KEYWORD, Default: 2


In [4]:
for name,param in inspect.signature(f) :
    print(name)
    print(param)

TypeError: 'Signature' object is not iterable